# df_brd_w_prd_v3

In [52]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [58]:
df_brd_w_prd_v2 = pd.read_csv("df_brd_w_prd_v2.csv")
df_brd_w_prd_v2 = df_brd_w_prd_v3.iloc[:, 1:]

In [70]:
df_brd_w_prd_v3 = df_brd_w_prd_v2.copy()

In [54]:
find_missing_df_brd_w_prd_v1_drop_lg = pd.read_csv("find_missing_df_brd_w_prd_v1_drop_lg_3220.csv")
find_missing_df_brd_w_prd_v1_drop_lg = find_missing_df_brd_w_prd_v1_drop_lg.iloc[:, 1:]

---
### lengh가 다른 브랜드의 product_link, product_name 재수집

In [56]:
exception = []

In [71]:
lst_brd_name = list(find_missing_df_brd_w_prd_v1_drop_lg.loc[find_missing_df_brd_w_prd_v1_drop_lg["crawling_prd_cnt"] != find_missing_df_brd_w_prd_v1_drop_lg["prd_len"]].brand_name)

In [57]:
for brd_name in tqdm(lst_brd_name):
    idx = df_brd_w_prd_v3.loc[df_brd_w_pr_v3['brand_name'] == i].index
    prd_link, prd_name = scrap_product_link(brd_name)
    df_brd_w_prd_v3.loc[idx, 'product_link'] = prd_link
    df_brd_w_prd_v3.loc[idx, 'product_name'] = prd_name

  0%|          | 0/125 [00:00<?, ?it/s]



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


고려은단
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


그램
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


글로우
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36


KeyboardInterrupt: 

In [55]:
def scrap_product_link(brd_name):
    
    try: 
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        print(userAgent)

        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument(f'user-agent={userAgent}')
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

        socket.setdefaulttimeout(30)

        url = 'https:/www.glowpick.com/searches/keywords'
        wd.get(url)
        time.sleep(5)

        wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
        wd.maximize_window()
        time.sleep(3)

        brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
        brd_input.send_keys(brd_name)
        brd_input.send_keys(Keys.ENTER)
        time.sleep(3)

        def parsing_html_with_class(parsing_info = '.products'):
            html = wd.page_source
            soup = soup = BeautifulSoup(html, "html.parser")
            interm_r = soup.select(parsing_info)
            return interm_r
        
        prd_link = []
        prd_name = []

        try:
            while True:

                product_lst = parsing_html_with_class()
                product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))
                
                #while wd.find_elements_by_class_name('details__brand')[product_len].text[:len(brd_name)] == brd_name:

                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)

                product_lst_2 = parsing_html_with_class()

                if product_lst == product_lst_2:
                    for i in range(product_len):
                        if (re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text).strip()) == brd_name:
                            code = re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0]))[i][1].split('-')[-1]
                            link = 'https://www.glowpick.com/products/' + code
                            prd_link.append(link)
                            prd_name.append(wd.find_elements_by_class_name('details__product')[i].text)
                    return prd_link, prd_name
                    break
                

        except NoSuchElementException:
            for i in range(product_len):
                if (re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text).strip()) == brd_name:
                    code = re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0]))[i][1].split('-')[-1]
                    link = 'https://www.glowpick.com/products/' + code
                    prd_link.append(link)
                    prd_name.append(wd.find_elements_by_class_name('details__product')[i].text)
            return prd_link, prd_name

    except Exception as e:
        exception.append(brd_name)
        print("brand name: {}".format(brd_name), e)
        pass